In [1]:
base_url = 'Files/sa_crime_datasets'

delta_table_name = 'crime_record_silver'
delta_table_path = f'Tables/{delta_table_name}'

StatementMeta(, 83f7c6be-f7e4-440d-aa44-f843b29d564c, 3, Finished, Available, Finished)

In [2]:
%run nb_02_sa_crime_record_wrangler

StatementMeta(, 83f7c6be-f7e4-440d-aa44-f843b29d564c, 4, Finished, Available, Finished)

In [3]:
record_schema = SACrimeRecordWrangler.create_init_schema()

StatementMeta(, 83f7c6be-f7e4-440d-aa44-f843b29d564c, 5, Finished, Available, Finished)

In [4]:
excel_df = SACrimeRecordWrangler.load_excel_files(spark, base_url, record_schema)
csv_df = SACrimeRecordWrangler.load_csv_files(spark, base_url, record_schema)

df = excel_df.union(csv_df)

StatementMeta(, 83f7c6be-f7e4-440d-aa44-f843b29d564c, 6, Finished, Available, Finished)

Call the methods on `SACrimeRecordWrangler`:
- `remove_all_na()`
- `reported_date_str_to_date()`
- `postcode_str_to_short()`
- `suburb_fill_null_empty()`
- `postcode_fill_null()`

In [5]:
df = SACrimeRecordWrangler.cleanse_df(df)

StatementMeta(, 83f7c6be-f7e4-440d-aa44-f843b29d564c, 7, Finished, Available, Finished)

In [6]:
from pyspark.sql.functions import when, count, col

df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).show()

StatementMeta(, 83f7c6be-f7e4-440d-aa44-f843b29d564c, 8, Finished, Available, Finished)

+------------+------+--------+------------+------------+--------------+-----+
|ReportedDate|Suburb|Postcode|LevelOneDesc|LevelTwoDesc|LevelThreeDesc|Count|
+------------+------+--------+------------+------------+--------------+-----+
|           0|     0|       0|           0|           0|             0|    0|
+------------+------+--------+------------+------------+--------------+-----+



In [7]:
SACrimeRecordWrangler.create_crime_records_silver_table(spark, delta_table_name)

StatementMeta(, 83f7c6be-f7e4-440d-aa44-f843b29d564c, 9, Finished, Available, Finished)

In [8]:
spark.sql(f'SELECT COUNT(*) FROM {delta_table_name}').show()

StatementMeta(, 83f7c6be-f7e4-440d-aa44-f843b29d564c, 10, Finished, Available, Finished)

+--------+
|count(1)|
+--------+
| 1391354|
+--------+



In [9]:
delta_table = DeltaTable.forPath(spark, delta_table_path)

SACrimeRecordWrangler.upsert_delta_table(delta_table, df)

StatementMeta(, 83f7c6be-f7e4-440d-aa44-f843b29d564c, 11, Finished, Available, Finished)

In [10]:
spark.sql(f'SELECT COUNT(*) FROM {delta_table_name}').show()

StatementMeta(, 83f7c6be-f7e4-440d-aa44-f843b29d564c, 12, Finished, Available, Finished)

+--------+
|count(1)|
+--------+
| 1391354|
+--------+

